In [148]:
import cplex
import pandas as pd
import re
import string

In [149]:
# parameter

X = list(range(12))
Y = list(range(5))
Z = list(range(3))
P = list(range(20))
T = list(range(11))

# 학생별 불가능한 시간을 담을 딕셔너리
FBD = {p:{x:{y:1 for y in Y}
             for x in X}
            for p in P}

# 많아야 2명 정도로 제한
MAX_NUM = {x:{y:{z:2 for z in Z}
              for y in Y}
           for x in X}

# 적어도 1명은 있어야 한다. 
MIN_NUM = {x:{y:{z:1 for z in Z}
              for y in Y}
           for x in X}

# 학생별 근무시간 : 학사 10시간 / 공로 5시간
Time = [5,5,5,10,10,5,5,5,5,5,
        10,5,10,10,10,10,5,10,10,5]

student = {
    0:'장동혁',
    1:'양희정',
    2:'정상석',
    3:'송윤범',
    4:'이승규', 
    5:'노기정',
    6:'최호경',
    7:'박지연',
    8:'정재건',
    9:'정의진',
    10:'김현아',
    11:'정경서',
    12:'이제리미',
    13:'김하나',
    14:'김동훈',
    15:'김형인',
    16:'한소희',
    17:'박현준',
    18:'차형주',
    19:'이승준'
}

# 강의실에 수업이 있는 시간
fbd_203 = [
    (0,1), #공학통계 월2
    (0,3), #공학통계 월4
    (2,4),(2,5),(2,6), #공학회계 수5,6,7
    (0,0),(4,0),(4,1), #자료구조 월1 금1,2
    (0,6),(1,6),(1,7), #기정시 월7 화7,8
    (1,5),(3,6),(3,7), #정보시스템 화6 목7,8
    (0,2),(2,2),(2,3) #딥러닝 월3 수3,4
          ]

M = 10000

In [150]:
# 기본 CSV 파일 만들기

# fbd_data_l = {}
# for p in P:
#     data = {}
#     for x in X:
#         for y in Y:
#             data[f'T{x}_D{y}'] = 1
#     fbd_data_l[f'{p}'] = data
    
# fbd_df = pd.DataFrame(fbd_data_l)

# fbd_df.to_csv('./fbd.csv')

In [151]:
# 불가능한 시간 데이터 받아오기

fbd_data = pd.read_csv('fbd_v2.csv')
fbd_data = fbd_data.iloc[:60,:]

for row in fbd_data.iterrows():
    time_info = row[1][0]
    x,y = [int(t) for t in re.findall('\d+',time_info)]
    for p in P:
        val = row[1][p+1]
        FBD[p][x][y] = val       

In [152]:
# 과사 최소,최대 시간 조정
office = 0

# 과사는 8교시까지만 한다.
for x in X[8:]:
    for y in Y:
        MAX_NUM[x][y][office] = 0
        MIN_NUM[x][y][office] = 0

# 1,3,4교시는 근무없다.
for x in [0,2,3]:
    for y in Y:
        MAX_NUM[x][y][office] = 0
        MIN_NUM[x][y][office] = 0

# 금요일은 근무 없다
for x in X:
    MAX_NUM[x][4][office] = 0
    MIN_NUM[x][4][office] = 0
        
# 5교시 이후에는 1시간만. 일단 최소인원 제약은 다 없애고 제약식에서 조건 추가.
for x in X[4:]:
    for y in Y:
        MIN_NUM[x][y][office] = 0
    

In [153]:
# 203 수업시간은 제외 -> 최소,최대 시간 0으로
for y,x in fbd_203:
    MAX_NUM[x][y][1] = 0
    MIN_NUM[x][y][1] = 0

In [194]:
#problem
problem = cplex.Cplex()

#variable

# 누가 무슨 요일 몇 교시에 어디에서 일하는지
problem.variables.add(names= [f"w_{p}_{x}_{y}_{z}" for p in P for x in X for y in Y for z in Z])

# 누가 무슨 요일 몇 교시부터 2시간 연달아서 일하는지
problem.variables.add(names= [f"sw_{p}_{t}_{y}_{z}" for p in P for t in T for y in Y for z in Z])

for p in P:
    for x in X:
        for y in Y:
            for z in Z: 
                problem.variables.set_types(f"w_{p}_{x}_{y}_{z}", problem.variables.type.binary)
    for t in T:
        for y in Y:
            for z in Z:
                problem.variables.set_types(f"sw_{p}_{t}_{y}_{z}", problem.variables.type.binary)
# for x in X:         
#     for y in Y:
#         problem.variables.set_types(f"wed_cov_{x}_{y}", problem.variables.type.binary)
    
#constraint

# 학사, 공로는 각각 5시간, 10시간을 채워야한다.
for p in P: 
    problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{x}_{y}_{z}" for x in X for y in Y for z in Z], 
                                   val= [1 for x in X for y in Y for z in Z])],
        rhs= [Time[p]],
        names= [f'time_const_{p}'],
        senses= ['E']
    )

# 각 학생들은 근무가 불가능한 시간이 있음.
for p in P:
    for x in X:
        for y in Y:
            problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[f"w_{p}_{x}_{y}_{z}" for z in Z], 
                                           val=[1 for z in Z])],
                rhs = [FBD[p][x][y]],
                names= [f'Forbidden_time_{p}_{x}_{y}'],
                senses= ['L']
            )

# 한 근무에 대해서 최대 인원이 있음.
for x in X:
    for y in Y:
        for z in Z:
            problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{x}_{y}_{z}" for p in P],
                                           val= [1 for p in P])],
                rhs = [MAX_NUM[x][y][z]],
                names= [f'Max_number_{x}_{y}_{z}'],
                senses= ['L']
            )

# 한 근무에 대해서 최소 인원이 있음.
for x in X:
    for y in Y:
        for z in Z:
            problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{x}_{y}_{z}" for p in P],
                                           val= [1 for p in P])],
                rhs = [MIN_NUM[x][y][z]],
                names= [f'Min_number_{x}_{y}_{z}'],
                senses= ['G']
            )
            
# 과사는 금요일 빼고 5교시 이후 1시간만
for y in Y[:4]:
    problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{x}_{y}_{0}" for p in P for x in X[4:]],
                                           val= [1 for p in P for x in X[4:]])],
                rhs = [1],
                names= [f'office_day_time_{y}'],
                senses= ['E']
            )

# 연달아 두시간 일하는 경우
for p in P:
    for t in T:
        for y in Y:
            for z in Z:
                problem.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(ind= [f"sw_{p}_{t}_{y}_{z}"] + [f"w_{p}_{t}_{y}_{z}" for z in Z] + [f"w_{p}_{t+1}_{y}_{z}" for z in Z] , 
                                               val= [2] + [-1 for z in Z] + [-1 for z in Z])],
                    rhs = [0],
                    names= [f'successive_work_{p}_{t}_{y}_{z}'],
                    senses= ['L']                 
                )
            

# 과사 2교시는 돌아가면서 => 1인 최대 1번
for p in P:
    problem.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{1}_{y}_{0}" for y in Y], 
                                       val= [1 for y in Y])],
            rhs = [1],
            names= [f'max_office_second_time_{y}'],
            senses= ['L']
        )

# 과사 근무는 한명씩만 한다.
for x in X:
    for y in Y:
        problem.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind= [f"w_{p}_{x}_{y}_{0}" for p in P], 
                                       val= [1 for p in P])],
            rhs = [1],
            names= [f'office_max_person_num_{y}'],
            senses= ['L']
        )

# 근무 마지막은 늘 연속되어야한다.
for t in T[8:]:
    for y in Y:
        problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind= [f"sw_{p}_{t}_{y}_{z}" for p in P for z in Z[1:]], 
                                   val= [1 for p in P for z in Z[1:]])],
        rhs = [1],
        names= [f'last_work_{t}_{y}_{z}'],
        senses= ['G']
    )
        
# objective
for p in P:
    for x in X:
        for y in Y:
            for z in Z: 
                problem.objective.set_linear([(f"w_{p}_{x}_{y}_{z}", 1)])
    for x in X[8:]:
        for y in Y:
            for z in Z: 
                problem.objective.set_linear([(f"w_{p}_{x}_{y}_{z}", 5)])
    
    for t in T:
        for y in Y:
            for z in Z:
                problem.objective.set_linear([(f"sw_{p}_{t}_{y}_{z}", -1)])
    
problem.objective.set_sense(problem.objective.sense.minimize)

# solve
problem.solve()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1


Tried aggregator 1 time.
MIP Presolve eliminated 3410 rows and 4006 columns.
MIP Presolve added 810 rows and 0 columns.
MIP Presolve modified 166 coefficients.
Reduced MIP has 2378 rows, 2389 columns, and 10717 nonzeros.
Reduced MIP has 2389 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (14.50 ticks)
Probing fixed 357 vars, tightened 0 bounds.
Probing time = 0.01 sec. (0.83 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 714 rows and 357 columns.
Reduced MIP has 1664 rows, 2032 columns, and 9247 nonzeros.
Reduced MIP has 2032 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (4.73 ticks)
Probing time = 0.00 sec. (1.25 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1664 rows, 2032 columns, and 9247 nonzeros.
Reduced MIP has 2032 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (7.67 ticks)
Probing time = 0.01 sec. (1.24 ticks)
Clique table members: 637.
MIP emphasis: balance opt

   1244     0      -24.0587   599      -16.0000      Cuts: 54   160389   50.28%
   1244     0      -24.0325   599      -16.0000      Cuts: 44   160486   50.20%
   1244     0      -24.0194   599      -16.0000      Cuts: 46   160574   50.12%
   1244     0      -23.9531   599      -16.0000      Cuts: 47   160762   49.71%
   1244     0      -23.9395   599      -16.0000      Cuts: 44   160846   49.62%
   1244     0      -23.9246   599      -16.0000      Cuts: 27   160949   49.53%
   1244     0      -23.9019   599      -16.0000      Cuts: 21   161031   49.39%
   1244     0      -23.8865   599      -16.0000      Cuts: 36   161120   49.29%
   1244     0      -23.8812   599      -16.0000      Cuts: 39   161219   49.26%
   1244     0      -23.8804   599      -16.0000      Cuts: 39   161272   49.25%
   1244     0      -23.8749   599      -16.0000      Cuts: 28   161365   49.22%
   1244     0      -23.8716   599      -16.0000      Cuts: 42   161459   49.20%
   1244     0      -23.8715   599      -

In [191]:
solution_loc_l = []

for z in Z:
    solution = [['' for y in Y]for x in X]
    for x in X:
        for y in Y:
            for p in P:
                if problem.solution.get_values(f"w_{p}_{x}_{y}_{z}") > 0.999:
                    if solution[x][y]:
                        solution[x][y] += ','+student[p]
                    else:
                        solution[x][y] += student[p]
                        
    solution_loc_l.append(solution)

In [192]:
office_df = pd.DataFrame(solution_loc_l[0])
room_203_df = pd.DataFrame(solution_loc_l[1])
room_538_df = pd.DataFrame(solution_loc_l[2])

In [193]:
room_538_df

,0,1,2,3,4
0,김형인,김하나,김하나,김하나,박지연
1,박현준,김하나,김하나,김하나,정상석
2,박현준,송윤범,김동훈,정경서,이승규
3,박현준,김현아,김동훈,정경서,노기정
4,박현준,김동훈,김현아,정경서,이승규
5,박현준,이승준,김현아,이제리미,이승규
6,박현준,이승준,김현아,차형주,이승규
7,한소희,이승준,김현아,최호경,이제리미
8,"양희정,김동훈",김형인,정재건,"장동혁,정상석","이승규,이제리미"
9,"김동훈,김형인","김하나,박현준","김현아,이제리미","최호경,차형주","이승규,이제리미"


In [183]:
room_203_df

,0,1,2,3,4
0,,"김현아,박현준",박지연,"송윤범,김하나",
1,,"김하나,박현준",장동혁,정경서,
2,,"송윤범,박현준",,"장동혁,송윤범",노기정
3,,"송윤범,이승준",,"송윤범,정경서",노기정
4,"정재건,차형주","김동훈,이승준",,"양희정,정경서","노기정,이제리미"
5,"송윤범,정재건",,,"양희정,정상석",이승규
6,,,,,"노기정,이제리미"
7,한소희,,정의진,,이제리미
8,한소희,김형인,정의진,양희정,이승규
9,한소희,김형인,김형인,최호경,이제리미


In [22]:
office_df.to_csv('./office_table.csv',encoding='utf-8-sig')
room_203_df.to_csv('./room_203_table.csv',encoding='utf-8-sig')
room_538_df.to_csv('./room_538_table.csv',encoding='utf-8-sig')

In [23]:
person_report = {}
for p in P:
    person_report[student[p]] = []
    for x in X:
        for y in Y:
            for z in Z:
                if problem.solution.get_values(f"w_{p}_{x}_{y}_{z}") > 0.999:
                    person_report[student[p]].append((x,y,z))

In [24]:
person_report

{'장동혁': [(1, 2, 1), (1, 3, 2), (2, 2, 2), (2, 3, 2), (3, 2, 2)],
 '양희정': [(4, 3, 2), (5, 3, 1), (6, 0, 0), (6, 3, 0), (7, 0, 2)],
 '정상석': [(1, 4, 2), (2, 4, 1), (4, 3, 1), (5, 3, 1), (6, 3, 2)],
 '송윤범': [(0, 3, 1),
  (1, 1, 0),
  (1, 3, 1),
  (2, 1, 1),
  (2, 3, 1),
  (3, 1, 2),
  (3, 3, 2),
  (9, 0, 1),
  (10, 0, 1),
  (10, 3, 2)],
 '이승규': [(2, 4, 2),
  (3, 4, 1),
  (4, 4, 2),
  (5, 4, 2),
  (6, 4, 1),
  (7, 4, 2),
  (8, 4, 2),
  (9, 4, 2),
  (10, 4, 2),
  (11, 4, 1)],
 '노기정': [(2, 4, 2), (3, 4, 2), (4, 4, 2), (5, 4, 1), (6, 4, 2)],
 '최호경': [(1, 0, 2), (2, 0, 2), (3, 0, 2), (4, 0, 2), (5, 0, 2)],
 '박지연': [(0, 4, 2), (1, 4, 2), (4, 4, 1), (5, 4, 2), (9, 4, 1)],
 '정재건': [(4, 3, 2), (5, 3, 2), (6, 3, 2), (7, 3, 2), (10, 3, 1)],
 '정의진': [(0, 2, 2), (1, 2, 0), (8, 0, 1), (8, 3, 1), (9, 3, 1)],
 '김현아': [(0, 1, 2),
  (1, 1, 1),
  (2, 1, 2),
  (3, 1, 2),
  (4, 1, 2),
  (4, 2, 2),
  (5, 1, 2),
  (5, 2, 2),
  (6, 2, 2),
  (7, 2, 2)],
 '정경서': [(1, 3, 2), (2, 3, 2), (3, 3, 1), (4, 3, 1), (5, 3, 2

In [203]:
# verification

fin_office_df = pd.read_csv('./office_table.csv').loc[:,'0':]
fin_203_df = pd.read_csv('./room_203_table.csv').loc[:,'0':]
fin_538_df = pd.read_csv('./room_538_table.csv').loc[:,'0':]

In [204]:
fin_office_df = fin_office_df.fillna('')
fin_203_df = fin_203_df.fillna('')
fin_538_df = fin_538_df.fillna('')

total_df_l = [fin_office_df, fin_203_df, fin_538_df]

In [205]:
final_person_report = {student[p]:[] for p in P}
for p in P:
    for z,df in enumerate(total_df_l):
        for x,row in enumerate(df.iterrows()):
            for y,cell in enumerate(row[1]):
                if student[p] in cell:
                    final_person_report[student[p]].append((x,y,z))


In [210]:
for p,val in final_person_report.items():
    if len(val) not in [5,10]:
        print(p, len(val))
    for idx,(x,y,z) in enumerate(val):
        for x2,y2,z2 in val[idx+1:]:
            if (x,y) == (x2,y2):
                print(p ,val)